In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from lightgbm import LGBMRegressor

# 데이터 로드

In [2]:
curr_dir = os.getcwd()

forders_tr = os.listdir(curr_dir+"\\data")

data = pd.read_csv(os.path.join(curr_dir+"\\data\\")+forders_tr[0])
data = data.drop(data.columns[0], axis=1)
data

submission = pd.read_csv('2-1_검증데이터셋.csv')
submission1 = submission.copy()          

# 전처리

In [121]:
soil = pd.read_csv('OBS_AAOS_TIM_20220802210224.csv', encoding='cp949')       # 외부 데이터 로드
stn = list(set(soil['지점'].values))

soil_df1 = []      # 일시 포함된 데이터 프레임
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    soil_df1.append(df)

    
soil_df = []        # 일시가 제외된 데이터 프레임 (추합을 위해)
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df = df.drop(['지점', '지점명', '일시'], axis=1)   
    soil_df.append(df) 
  
day = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')    # 21.11.26~22.03.20
day
day1 = pd.DataFrame(day)
day1.columns = ['일자']
day1
day1.to_csv('day_real.csv', index=False)


for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df.to_csv(os.path.join(str(stn[i])+'.csv'), index=False)   # 관측소마다.csv 파일로 저장

In [251]:
# 관측소마다 일시(일자) 통합을 위해 'day_real'파일과 위에서 저장된 모든 관측소에 대해 엑셀 VLOOKUP 함수 이용하여 
# 중간에 비어 있는 일시를 채워주는 작업 후, 각 파일을 _1.csv 파일로 저장

soil_df2 = []
for i in range(len(stn)):
    df = pd.read_csv(os.path.join(str(stn[i])+'_1.csv'), encoding='cp949')
    df = df.drop('일자', axis=1)
    df = df.fillna(0)
    soil_df2.append(df)



soil_df3 = soil_df2[0].copy()
for i in range(len(soil_df[0].columns)):
    soil_df2[0][soil_df2[0].columns[i]] = 0

# 각 관측소마다 평균 내기 (추합)
for i in range(1, len(soil_df2)):
    soil_df3 = soil_df3 + soil_df2[i]
soil_df3 = soil_df3/len(soil_df2)
soil_df3['일자'] = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')


# 제공 데이터와의 통합을 위해 제공 데이터에 없는 일시를 외부 데이터에서 제외시킴
d1 = soil_df3[soil_df3['일자'] == '2021-12-22 0:00'].index[0]
for i in range(d1, d1+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)

d2 = soil_df3[soil_df3['일자'] == '2022-03-01 0:00'].index[0]
for i in range(d2, d2+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d3 = soil_df3[soil_df3['일자'] == '2022-03-14 0:00'].index[0]
for i in range(d3, d3+24):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d4 = soil_df3[soil_df3['일자'] == '2022-03-16 0:00'].index[0]
for i in range(d4, d4+24*2):
    soil_df3 = soil_df3.drop(i,axis=0)

soil_df3 = soil_df3.reset_index()
soil_df3 = soil_df3.drop('index', axis=1)

# 제공 데이터와 외부 데이터 통합 (시간 데이터 제외 모든 열)
col = ['smart_farm.insolation', 'smart_farm.out_tmperature',\
       'smart_farm.out_humidity', 'smart_farm.wind_speed',\
       'smart_farm.wind_direction', 'smart_farm.in_tmperature',\
       'smart_farm.in_humidity', 'smart_farm.shield_light_h',\
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',\
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',\
       'smart_farm.ceiling', 'smart_farm.floating_fan',\
       'smart_farm.fan_coil_b_site',\
       'smart_farm.ventilation_temperature_control',\
       'smart_farm.heating_temperature_set_up']          

data_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(data)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(data[col[k]][60*i:60*(i+1)]))
    data_[col[k]] = h
    
h1 = []
for i in range(int(len(data)/60)):
    h1.append(np.sum(data['smart_farm.heat_supply'][60*i:60*(i+1)]))
data_['smart_farm.heat_supply'] = h1



real_data = pd.concat([soil_df3, data_], axis=1)
real_data1 = pd.concat([real_data[real_data.columns[4]], real_data[real_data.columns[:4]], real_data[real_data.columns[5:]]], axis=1)
real_data1 = real_data1.reset_index()
real_data1 = real_data1.drop('index', axis=1)
real_data1 = real_data1.dropna(subset = ['일자'])
real_data1.to_csv('real_data_통합.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.86it/s]


In [262]:
# 위에서 저장한 'real_data_통합' 파일과 'day_real' 파일 이용하여
# 엑셀 VLOOKUP 함수 이용하여 21.11.26부터 22.03.20까지 비어 일자 채운 후, 다시 'real_data_결측' 파일로 저장

pre = pd.read_csv('real_data_결측.csv', encoding='cp949')
pre1 = pre.interpolate(method ='linear')      
pre1 = pre1.dropna(subset = ['일자'])
pre1.to_csv('real_data_최종.csv', index=False)  # 최종 전처리 파일(결측치 제거한)

real1 = pd.read_csv('real_data_최종.csv', encoding='utf-8')
real1

# 자른 이유? 20개를 추출하기 위해 일자 슬라이싱이 필요함.
training = real1[:-504]
testing = real1[-504:]
testing = testing.reset_index()
testing = testing.drop('index', axis=1)

In [3]:
real1 = pd.read_csv('real_data_최종.csv', encoding='utf-8')
real1

# 자른 이유? 20개를 추출하기 위해 일자 슬라이싱이 필요함.
training = real1[:-504]
testing = real1[-504:]
testing = testing.reset_index()
testing = testing.drop('index', axis=1)

# 모델 1

In [4]:
max_depth = 5
n_estimators = 5000             # 400  #1000
lr = 0.01             # 0.017
num_leaves = 100
early_stopping_rounds = 300      # 100
verbose = True                  # True

In [7]:
X = training.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=False)  

X_test= testing.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)

lgbm_reg = LGBMRegressor(objective= "regression",
                     max_depth= max_depth,
                     n_estimators= n_estimators,
                     learning_rate= lr,
                     num_leaves = num_leaves)
    
lgbm_reg.fit(X_train, y_train,
          eval_set=[(X_val, y_val)],
          eval_metric=["mse"],
          early_stopping_rounds=early_stopping_rounds,
          verbose=verbose) 

C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[1]	valid_0's l2: 210.915
[2]	valid_0's l2: 209.141
[3]	valid_0's l2: 207.402
[4]	valid_0's l2: 205.698
[5]	valid_0's l2: 204.012
[6]	valid_0's l2: 202.381
[7]	valid_0's l2: 200.761
[8]	valid_0's l2: 199.199
[9]	valid_0's l2: 197.69
[10]	valid_0's l2: 196.195
[11]	valid_0's l2: 194.802
[12]	valid_0's l2: 193.436
[13]	valid_0's l2: 192.047
[14]	valid_0's l2: 190.71
[15]	valid_0's l2: 189.378
[16]	valid_0's l2: 188.131
[17]	valid_0's l2: 186.852
[18]	valid_0's l2: 185.647
[19]	valid_0's l2: 184.409
[20]	valid_0's l2: 183.243
[21]	valid_0's l2: 182.038
[22]	valid_0's l2: 180.915
[23]	valid_0's l2: 179.781
[24]	valid_0's l2: 178.704
[25]	valid_0's l2: 177.553
[26]	valid_0's l2: 176.525
[27]	valid_0's l2: 175.494
[28]	valid_0's l2: 174.504
[29]	valid_0's l2: 173.502
[30]	valid_0's l2: 172.224
[31]	valid_0's l2: 171.23
[32]	valid_0's l2: 170.299
[33]	valid_0's l2: 169.079
[34]	valid_0's l2: 168.125
[35]	valid_0's l2: 167.23
[36]	valid_0's l2: 166.364
[37]	valid_0's l2: 165.215
[38]	valid_0's

[512]	valid_0's l2: 108.046
[513]	valid_0's l2: 108.044
[514]	valid_0's l2: 108.054
[515]	valid_0's l2: 108.058
[516]	valid_0's l2: 108.066
[517]	valid_0's l2: 108.054
[518]	valid_0's l2: 108.071
[519]	valid_0's l2: 108.064
[520]	valid_0's l2: 108.091
[521]	valid_0's l2: 108.097
[522]	valid_0's l2: 108.092
[523]	valid_0's l2: 108.078
[524]	valid_0's l2: 108.082
[525]	valid_0's l2: 108.063
[526]	valid_0's l2: 108.05
[527]	valid_0's l2: 108.023
[528]	valid_0's l2: 108.029
[529]	valid_0's l2: 107.979
[530]	valid_0's l2: 107.985
[531]	valid_0's l2: 108.009
[532]	valid_0's l2: 108.003
[533]	valid_0's l2: 108.029
[534]	valid_0's l2: 107.963
[535]	valid_0's l2: 107.928
[536]	valid_0's l2: 107.915
[537]	valid_0's l2: 107.917
[538]	valid_0's l2: 107.931
[539]	valid_0's l2: 107.908
[540]	valid_0's l2: 107.91
[541]	valid_0's l2: 107.879
[542]	valid_0's l2: 107.819
[543]	valid_0's l2: 107.808
[544]	valid_0's l2: 107.782
[545]	valid_0's l2: 107.788
[546]	valid_0's l2: 107.719
[547]	valid_0's l2: 10

LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=5000,
              num_leaves=100, objective='regression')

In [9]:
pred_ = lgbm_reg.predict(X_test)
h1 = []
for i in range(int(len(pred_)/24)):
    h1.append(np.sum(pred_[24*i:24*(i+1)]))
h1
# 4~13, 18~20
h1.insert(0,0)     # 한 번만 클릭!
h1

h11 = h1[4:14] + h1[18:21]
h11
submission1['heat_supply_day1'] = h11
submission1 

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,748.337751,NaN
1,2022,3,5,425.712588,NaN
2,2022,3,6,446.478205,NaN
3,2022,3,7,502.962061,NaN
4,2022,3,8,760.501280,NaN
5,2022,3,9,495.286703,NaN
6,2022,3,10,473.227582,NaN
7,2022,3,11,420.783625,NaN
8,2022,3,12,421.237132,NaN
9,2022,3,13,344.155403,NaN


# 모델 2(절감 모델)

In [10]:
X = training.drop(['일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=False)   # shuffle은 안 됨. 시간적순서라

X_test = testing.drop(['일자','smart_farm.heat_supply'], axis=1)


lgbm_reg = LGBMRegressor(objective= "regression",
                     max_depth= max_depth,
                     n_estimators= n_estimators,
                     learning_rate= lr,
                     num_leaves = num_leaves)
    
lgbm_reg.fit(X_train, y_train,
          eval_set=[(X_val, y_val)],
          eval_metric=["mse"],
          early_stopping_rounds=early_stopping_rounds,
          verbose=verbose) 

C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[1]	valid_0's l2: 211.351
[2]	valid_0's l2: 209.838
[3]	valid_0's l2: 208.519
[4]	valid_0's l2: 207.063
[5]	valid_0's l2: 205.625
[6]	valid_0's l2: 204.305
[7]	valid_0's l2: 202.552
[8]	valid_0's l2: 201.258
[9]	valid_0's l2: 199.597
[10]	valid_0's l2: 198.037
[11]	valid_0's l2: 196.45
[12]	valid_0's l2: 195.037
[13]	valid_0's l2: 193.555
[14]	valid_0's l2: 191.997
[15]	valid_0's l2: 190.903
[16]	valid_0's l2: 189.465
[17]	valid_0's l2: 188.08
[18]	valid_0's l2: 186.815
[19]	valid_0's l2: 185.791
[20]	valid_0's l2: 184.462
[21]	valid_0's l2: 183.166
[22]	valid_0's l2: 181.873
[23]	valid_0's l2: 180.858
[24]	valid_0's l2: 179.72
[25]	valid_0's l2: 178.815
[26]	valid_0's l2: 177.461
[27]	valid_0's l2: 176.261
[28]	valid_0's l2: 175.221
[29]	valid_0's l2: 174.193
[30]	valid_0's l2: 172.917
[31]	valid_0's l2: 171.758
[32]	valid_0's l2: 170.998
[33]	valid_0's l2: 169.777
[34]	valid_0's l2: 168.873
[35]	valid_0's l2: 167.832
[36]	valid_0's l2: 166.954
[37]	valid_0's l2: 166.276
[38]	valid_0'

[478]	valid_0's l2: 104.937
[479]	valid_0's l2: 104.929
[480]	valid_0's l2: 104.948
[481]	valid_0's l2: 104.956
[482]	valid_0's l2: 104.96
[483]	valid_0's l2: 104.966
[484]	valid_0's l2: 104.957
[485]	valid_0's l2: 104.967
[486]	valid_0's l2: 104.972
[487]	valid_0's l2: 104.969
[488]	valid_0's l2: 104.998
[489]	valid_0's l2: 104.99
[490]	valid_0's l2: 104.98
[491]	valid_0's l2: 104.985
[492]	valid_0's l2: 104.981
[493]	valid_0's l2: 104.991
[494]	valid_0's l2: 104.996
[495]	valid_0's l2: 105.003
[496]	valid_0's l2: 104.996
[497]	valid_0's l2: 105.004
[498]	valid_0's l2: 105
[499]	valid_0's l2: 105.031
[500]	valid_0's l2: 105.031
[501]	valid_0's l2: 105.037
[502]	valid_0's l2: 105.044
[503]	valid_0's l2: 105.049
[504]	valid_0's l2: 105.06
[505]	valid_0's l2: 105.088
[506]	valid_0's l2: 105.098
[507]	valid_0's l2: 105.105
[508]	valid_0's l2: 105.133
[509]	valid_0's l2: 105.141
[510]	valid_0's l2: 105.132
[511]	valid_0's l2: 105.14
[512]	valid_0's l2: 105.144
[513]	valid_0's l2: 105.152
[

LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=5000,
              num_leaves=100, objective='regression')

In [11]:
pred_2 = lgbm_reg.predict(X_test)
h2 = []
for i in range(int(len(pred_2)/24)):
    h2.append(np.sum(pred_2[24*i:24*(i+1)]))
h2
# 4~13, 18~20
h2.insert(0,0)     # 한 번만 클릭!
h2

h22 = h2[4:14] + h2[18:21]
h22
submission1['heat_supply_day2'] = h22
submission1 

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,748.337751,699.033956
1,2022,3,5,425.712588,399.213489
2,2022,3,6,446.478205,461.623375
3,2022,3,7,502.962061,461.365036
4,2022,3,8,760.501280,661.375174
5,2022,3,9,495.286703,435.095738
6,2022,3,10,473.227582,404.602804
7,2022,3,11,420.783625,408.104910
8,2022,3,12,421.237132,403.430272
9,2022,3,13,344.155403,362.669209


In [12]:
submission1.to_csv('220224_최종코드틀_LGBM.csv', index=False) 